In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [ ]:
# Additional dependencies for this notebook
!pip install -q gem-suite

*הערכת שימוש: 3 דקות על מעבד Heron r2 (הערה: זוהי הערכה בלבד. זמן הריצה שלך עשוי להשתנות.)*

## רקע

מדריך זה מדגים כיצד להגשים מעבר פאזה של Nishimori על מעבד קוונטי של IBM&reg;. ניסוי זה תואר במקור במאמר [*Realizing the Nishimori transition across the error threshold for constant-depth quantum circuits*](https://arxiv.org/abs/2309.02863).

מעבר הפאזה של Nishimori מתייחס למעבר בין פאזות מסודרות בטווח קצר לטווח ארוך במודל Ising עם קשרים אקראיים. במחשב קוונטי, הפאזה המסודרת בטווח ארוך באה לידי ביטוי במצב בו קיוביטים משולבים קוונטית על פני כל המכשיר. מצב משולב קוונטית זה מוכן באמצעות פרוטוקול *generation of entanglement by measurement* (GEM). באמצעות מדידות אמצע-מעגל, פרוטוקול GEM מסוגל לשלב קוונטית קיוביטים על פני כל המכשיר באמצעות מעגלים בעומק קבוע בלבד. מדריך זה משתמש במימוש של פרוטוקול GEM מחבילת התוכנה [GEM Suite](https://github.com/qiskit-community/gem-suite).

## דרישות

לפני שתתחילו במדריך זה, ודאו שיש לכם את התוכנות הבאות מותקנות:

- Qiskit SDK v1.0 ומעלה, עם תמיכה ב-[visualization](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime v0.22 ומעלה ( `pip install qiskit-ibm-runtime` )
- GEM Suite ( `pip install gem-suite` )

## הכנה

In [2]:
import matplotlib.pyplot as plt

from collections import defaultdict

from qiskit_ibm_runtime import QiskitRuntimeService

from qiskit.transpiler import generate_preset_pass_manager

from gem_suite import PlaquetteLattice
from gem_suite.experiments import GemExperiment

## שלב 1: מיפוי קלטים קלאסיים לבעיה קוונטית
פרוטוקול GEM פועל על מעבד קוונטי עם קישוריות קיוביטים המתוארת על ידי סריג. מעבדים קוונטיים של IBM כיום משתמשים ב-[heavy hex lattice](https://www.ibm.com/quantum/blog/heavy-hex-lattice). הקיוביטים של המעבד מקובצים ל-*פלאקטות* (plaquettes) על בסיס תא היחידה של הסריג בו הם שוכנים. מכיוון שקיוביט עשוי להופיע ביותר מתא יחידה אחד, הפלאקטות אינן זרות. בסריג heavy hex, פלאקט מכיל 12 קיוביטים. הפלאקטות עצמן גם יוצרות סריג, שבו שתי פלאקטות מחוברות אם הן חולקות קיוביטים כלשהם. בסריג heavy hex, פלאקטות שכנות חולקות 3 קיוביטים.

בחבילת התוכנה GEM Suite, המחלקה הבסיסית למימוש פרוטוקול GEM היא `PlaquetteLattice`, המייצגת את סריג הפלאקטות (שונה מסריג heavy hex). ניתן לאתחל `PlaquetteLattice` ממפת צימוד קיוביטים. כרגע, רק מפות צימוד heavy hex נתמכות.

תא הקוד הבא מאתחל סריג פלאקטות ממפת הצימוד של מעבד קוונטי של IBM. סריג הפלאקטות לא תמיד מקיף את כל החומרה. לדוגמה, ל-`ibm_torino` יש 133 קיוביטים בסך הכל, אך סריג הפלאקטות הגדול ביותר שמתאים למכשיר משתמש רק ב-125 מהם, ומכיל סך של 18 פלאקטות. דבר דומה ניתן לראות גם במכשירי IBM Quantum&reg; עם ספירות קיוביטים שונות.

In [ ]:
# QiskitRuntimeService.save_account(channel="ibm_quantum", token="<YOUR_API_KEYN>", overwrite=True, set_as_default=True)
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
plaquette_lattice = PlaquetteLattice.from_coupling_map(backend.coupling_map)

print(f"Number of qubits in backend: {backend.num_qubits}")
print(
    f"Number of qubits in plaquette lattice: {len(list(plaquette_lattice.qubits()))}"
)
print(f"Number of plaquettes: {len(list(plaquette_lattice.plaquettes()))}")

Number of qubits in backend: 133
Number of qubits in plaquette lattice: 125
Number of plaquettes: 18


You can visualize the plaquette lattice by generating a diagram of its graph representation. In the diagram, the plaquettes are represented by labeled hexagons, and two plaquettes are connected by an edge if they share qubits.

In [7]:
plaquette_lattice.draw_plaquettes()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/625882a4-faeb-4d96-b441-c989f43c4dea-0.avif" alt="Output of the previous code cell" />

ניתן לדמיין את סריג הפלאקטות על ידי יצירת דיאגרמה של ייצוג הגרף שלו. בדיאגרמה, הפלאקטות מיוצגות על ידי משושים מסומנים, ושתי פלאקטות מחוברות בקצה אם הן חולקות קיוביטים.

In [8]:
# Get a list of the plaquettes
plaquettes = list(plaquette_lattice.plaquettes())
# Display information about plaquette 0
plaquettes[0]

PyPlaquette(index=0, qubits=[0, 1, 2, 3, 4, 15, 16, 19, 20, 21, 22, 23], neighbors=[3, 1])

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/625882a4-faeb-4d96-b441-c989f43c4dea-0.avif)

ניתן לאחזר מידע על פלאקטות בודדות, כגון הקיוביטים שהן מכילות, באמצעות המתודה `plaquettes`.

In [9]:
plaquette_lattice.draw_qubits()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/a19d63ce-3572-4081-a008-c1332fbbe303-0.avif" alt="Output of the previous code cell" />

In addition to the qubit labels and the edges indicating which qubits are connected, the diagram contains three additional pieces of information that are relevant to the GEM protocol:
- Each qubit is either shaded (gray) or unshaded. The shaded qubits are "site" qubits that represent the sites of the Ising model, and the unshaded qubits are "bond" qubits used to mediate interactions between the site qubits.
- Each site qubit is labeled either (A) or (B), indicating one of two roles a site qubit can play in the GEM protocol (the roles are explained later).
- Each edge is colored using one of six colors, thus partitioning the edges into six groups. This partitioning determines how two-qubit gates can be parallelized, as well as different scheduling patterns that are likely to incur different amounts of error on a noisy quantum processor. Because edges in a group are disjoint, a layer of two-qubit gates can be applied on those edges simultaneously. In fact, it is possible to partition the six colors into three groups of two colors such that the union of each group of two colors is still disjoint. Therefore, only three layers of two-qubit gates are needed to activate every edge. There are 12 ways to so partition the six colors, and each such partition yields a different 3-layer gate schedule.

Now that you have created a plaquette lattice, the next step is to initialize a `GemExperiment` object, passing both the plaquette lattice and the backend that you intend to run the experiment on. The `GemExperiment` class manages the actual implementation of the GEM protocol, including generating circuits, submitting jobs, and analyzing the data. The following code cell initializes the experiment class while restricting the plaquette lattice to only two of the plaquettes (21 qubits), reducing the size of the experiment to ensure that the noise in the hardware doesn't overwhelm the signal.

In [16]:
gem_exp = GemExperiment(plaquette_lattice.filter([9, 12]), backend=backend)

# visualize the plaquette lattice after filtering
plaquette_lattice.filter([9, 12]).draw_qubits()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/02357c6e-5c83-4ac0-811d-22602d9f33d5-0.avif" alt="Output of the previous code cell" />

ניתן גם לייצר דיאגרמה של הקיוביטים הבסיסיים שיוצרים את סריג הפלאקטות.

In [12]:
circuits = gem_exp.circuits()
print(f"Total number of circuits: {len(circuits)}")

Total number of circuits: 252


![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/a19d63ce-3572-4081-a008-c1332fbbe303-0.avif)

בנוסף לתוויות הקיוביטים והקצוות המציינים אילו קיוביטים מחוברים, הדיאגרמה מכילה שלושה פיסות מידע נוספות הרלוונטיות לפרוטוקול GEM:
- כל קיוביט הוא מוצל (אפור) או לא מוצל. הקיוביטים המוצללים הם קיוביטי "אתר" (site) המייצגים את האתרים של מודל Ising, והקיוביטים הלא מוצללים הם קיוביטי "קשר" (bond) המשמשים לתיווך אינטראקציות בין קיוביטי האתר.
- כל קיוביט אתר מסומן כ-(A) או (B), המציין אחד משני תפקידים שקיוביט אתר יכול למלא בפרוטוקול GEM (התפקידים מוסברים מאוחר יותר).
- כל קצה צבוע באחד משישה צבעים, ובכך מחלק את הקצוות לשש קבוצות. חלוקה זו קובעת כיצד ניתן לבצע שערים דו-קיוביטיים במקביל, כמו גם תבניות תזמון שונות שסביר שיגרמו לכמויות שונות של שגיאה על מעבד קוונטי רועש. מכיוון שקצוות בקבוצה זרים, ניתן להחיל שכבת שערים דו-קיוביטיים על אותם קצוות בו-זמנית. למעשה, ניתן לחלק את ששת הצבעים לשלוש קבוצות של שני צבעים כך שהאיחוד של כל קבוצה של שני צבעים עדיין זר. לכן, יש צורך רק בשלוש שכבות של שערים דו-קיוביטיים כדי להפעיל כל קצה. ישנן 12 דרכים לחלק את ששת הצבעים כך, וכל חלוקה כזו מניבה לוח זמנים שונה של 3 שכבות שערים.

כעת לאחר שיצרתם סריג פלאקטות, השלב הבא הוא לאתחל אובייקט `GemExperiment`, תוך העברת סריג הפלאקטות וה-backend שבו אתם מתכננים להריץ את הניסוי. המחלקה `GemExperiment` מנהלת את המימוש בפועל של פרוטוקול GEM, כולל יצירת מעגלים, הגשת עבודות וניתוח הנתונים. תא הקוד הבא מאתחל את מחלקה הניסוי תוך הגבלת סריג הפלאקטות לשתיים בלבד מהפלאקטות (21 קיוביטים), מצמצם את גודל הניסוי כדי להבטיח שהרעש בחומרה לא יעלה על האות.

In [13]:
# Restrict experiment to the first scheduling pattern
gem_exp.set_experiment_options(schedule_idx=0)

# There are less circuits now
circuits = gem_exp.circuits()
print(f"Total number of circuits: {len(circuits)}")

# Print the RZZ angles swept over
print(f"RZZ angles:\n{gem_exp.parameters()}")

Total number of circuits: 21
RZZ angles:
[0.         0.07853982 0.15707963 0.23561945 0.31415927 0.39269908
 0.4712389  0.54977871 0.62831853 0.70685835 0.78539816 0.86393798
 0.9424778  1.02101761 1.09955743 1.17809725 1.25663706 1.33517688
 1.41371669 1.49225651 1.57079633]


![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/02357c6e-5c83-4ac0-811d-22602d9f33d5-0.avif)

מעגל פרוטוקול GEM נבנה באמצעות השלבים הבאים:
1. הכנת מצב all-$|+\rangle$ על ידי החלת שער Hadamard על כל קיוביט.
2. החלת שער $R_{ZZ}$ בין כל זוג קיוביטים מחוברים. ניתן להשיג זאת באמצעות 3 שכבות של שערים. כל שער $R_{ZZ}$ פועל על קיוביט אתר וקיוביט קשר. אם קיוביט האתר מסומן (B), אז הזווית קבועה ל-$\frac{\pi}{2}$. אם קיוביט האתר מסומן (A), אז הזווית מורשית להשתנות, מייצרת מעגלים שונים. כברירת מחדל, טווח הזוויות מוגדר ל-21 נקודות מרווחות שווה בין $0$ ל-$\frac{\pi}{2}$, כולל.
3. מדידת כל קיוביט קשר בבסיס Pauli $X$. מכיוון שקיוביטים נמדדים בבסיס Pauli $Z$, ניתן לבצע זאת על ידי החלת שער Hadamard לפני מדידת הקיוביט.

שימו לב שהמאמר המצוטט בהקדמה למדריך זה משתמש באמנה שונה לזווית $R_{ZZ}$, השונה מהאמנה המשמשת במדריך זה בפקטור של 2.

בשלב 3, רק קיוביטי הקשר נמדדים. כדי להבין באיזה מצב נשארים קיוביטי האתר, מועיל לשקול את המקרה שבו זווית $R_{ZZ}$ המוחלת על קיוביטי אתר (A) בשלב 2 שווה ל-$\frac{\pi}{2}$. במקרה זה, קיוביטי האתר נשארים במצב משולב קוונטית מאוד הדומה למצב GHZ,

$$
\lvert \text{GHZ} \rangle = \lvert 00 \cdots 00 \rangle + \lvert 11 \cdots 11 \rangle.
$$

בשל האקראיות בתוצאות המדידה, המצב בפועל של קיוביטי האתר עשוי להיות מצב שונה עם סדר ארוך טווח, לדוגמה, $\lvert 00110 \rangle + \lvert 11001 \rangle$. עם זאת, ניתן לשחזר את מצב GHZ על ידי החלת פעולת פענוח המבוססת על תוצאות המדידה. כאשר זווית $R_{ZZ}$ מכוונת כלפי מטה מ-$\frac{\pi}{2}$, ניתן עדיין לשחזר את הסדר ארוך הטווח עד לזווית קריטית, שבהיעדר רעש, היא בקירוב $0.3 \pi$. מתחת לזווית זו, המצב המתקבל כבר אינו מציג שזירה ארוכת טווח. מעבר זה בין נוכחות והיעדר סדר ארוך טווח הוא מעבר הפאזה של Nishimori.

בתיאור לעיל, קיוביטי האתר נותרו ללא מדידה, וניתן לבצע את פעולת הפענוח על ידי החלת שערים קוונטיים. בניסוי כפי שמיושם בחבילת GEM, שמדריך זה עוקב אחריו, קיוביטי האתר נמדדים למעשה, ופעולת הפענוח מוחלת בשלב עיבוד קלאסי לאחר מכן.

בתיאור לעיל, ניתן לבצע את פעולת הפענוח על ידי החלת שערים קוונטיים על קיוביטי האתר כדי לשחזר את המצב הקוונטי. עם זאת, אם המטרה היא למדוד מיד את המצב, לדוגמה למטרות אפיון, אז קיוביטי האתר נמדדים יחד עם קיוביטי הקשר, וניתן להחיל את פעולת הפענוח בשלב עיבוד קלאסי לאחר מכן. כך מיושם הניסוי בחבילת GEM, שמדריך זה עוקב אחריו.

בנוסף להיותו תלוי בזווית $R_{ZZ}$ בשלב 2, שכברירת מחדל סוקרת 21 ערכים, מעגל פרוטוקול GEM תלוי גם בתבנית התזמון המשמשת ליישום 3 השכבות של שערי $R_{ZZ}$. כפי שנדון קודם לכן, ישנן 12 תבניות תזמון כאלה. לכן, המספר הכולל של מעגלים בניסוי הוא $21 \times 12 = 252$.

ניתן ליצור את המעגלים של הניסוי באמצעות המתודה `circuits` של המחלקה `GemExperiment`.

In [14]:
# Get the circuit at index 5
circuit = circuits[5]
# Remove the final measurements to ease visualization
circuit.remove_final_measurements()
# Draw the circuit
circuit.draw("mpl", fold=-1, scale=0.5)

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/fd57d483-c70b-4ad5-b309-15750ad38bac-0.avif" alt="Output of the previous code cell" />

## Step 2: Optimize problem for quantum hardware execution

Transpiling quantum circuits for execution on hardware typically involves a [number of stages](/docs/guides/transpiler-stages). Typically, the stages that incur the most computational overhead are choosing the qubit layout, routing the two-qubit gates to conform to the qubit connectivity of the hardware, and optimizing the circuit to minimize its gate count and depth. In the GEM protocol, the layout and routing stages are unnecessary because the hardware connectivity is already incorporated into the design of the protocol. The circuits already have a qubit layout, and the two-qubit gates are already mapped onto native connections. Furthermore, in order to preserve the structure of the circuit as the $R_{ZZ}$ angle is varied, only very basic circuit optimization should be performed.

The `GemExperiment` class transparently transpiles circuits when executing the experiment. The layout and routing stages are already overridden by default to do nothing, and circuit optimization is performed at a level that only optimizes single-qubit gates. However, you can override or pass additional options using the `set_transpile_options` method. For the sake of visualization, the following code cell manually transpiles the circuit displayed previously, and draws the transpiled circuit.

In [15]:
# Demonstrate setting transpile options
gem_exp.set_transpile_options(
    optimization_level=1  # This is the default optimization level
)
pass_manager = generate_preset_pass_manager(
    backend=backend,
    initial_layout=list(gem_exp.physical_qubits),
    **dict(gem_exp.transpile_options),
)
transpiled = pass_manager.run(circuit)
transpiled.draw("mpl", idle_wires=False, fold=-1, scale=0.5)

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/e9b99d48-8d33-46b5-bff5-480ab1c1c1f2-0.avif" alt="Output of the previous code cell" />

למטרות מדריך זה, מספיק לשקול רק תבנית תזמון בודדת. תא הקוד הבא מגביל את הניסוי לתבנית התזמון הראשונה. כתוצאה מכך, לניסוי יש רק 21 מעגלים, אחד לכל זווית $R_{ZZ}$ שעליה מעבירים.

In [10]:
exp_data = gem_exp.run(shots=10_000)

To wait for the results, call the `block_for_results` method of the `ExperimentData` object. This call will cause the interpreter to hang until the jobs are finished.

In [11]:
exp_data.block_for_results()

ExperimentData(GemExperiment, d0d5880a-34c1-4aab-a7b6-c4f58516bc03, job_ids=['cwg12ptmptp00082khhg'], metadata=<5 items>, figure_names=['two_point_correlation.svg', 'normalized_variance.svg', 'plaquette_ops.svg', 'bond_ops.svg'])

תא הקוד הבא מצייר דיאגרמה של המעגל באינדקס 5. כדי לצמצם את גודל הדיאגרמה, שערי המדידה בסוף המעגל מוסרים.

In [ ]:
def magnetization_distribution(
    counts_dict: dict[str, int],
) -> dict[str, float]:
    """Compute magnetization distribution from counts dictionary."""
    # Construct dictionary from magnetization to count
    mag_dist = defaultdict(float)
    for bitstring, count in counts_dict.items():
        mag = bitstring.count("0") - bitstring.count("1")
        mag_dist[mag] += count
    # Normalize
    shots = sum(counts_dict.values())
    for mag in mag_dist:
        mag_dist[mag] /= shots
    return mag_dist


# Get counts dictionaries with and without decoding
data = exp_data.data()
# Get the last data point, which is at the angle for the GHZ state
raw_counts = data[-1]["counts"]
# Without decoding
site_indices = [
    i for i, q in enumerate(gem_exp.plaquettes.qubits()) if q.role == "Site"
]
site_raw_counts = defaultdict(int)
for key, val in raw_counts.items():
    site_str = "".join(key[-1 - i] for i in site_indices)
    site_raw_counts[site_str] += val
# With decoding
_, site_decoded_counts = gem_exp.plaquettes.decode_outcomes(
    raw_counts, return_counts=True
)

# Compute magnetization distribution
raw_magnetization = magnetization_distribution(site_raw_counts)
decoded_magnetization = magnetization_distribution(site_decoded_counts)

# Plot
plt.bar(*zip(*raw_magnetization.items()), label="raw")
plt.bar(*zip(*decoded_magnetization.items()), label="decoded", width=0.3)
plt.legend()
plt.xlabel("Magnetization")
plt.ylabel("Frequency")
plt.title("Magnetization distribution with and without decoding")

Text(0.5, 1.0, 'Magnetization distribution with and without decoding')

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/8ead3582-16df-4616-836c-bdce867ad6b8-1.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/fd57d483-c70b-4ad5-b309-15750ad38bac-0.avif)

## שלב 2: אופטימיזציה של הבעיה עבור הרצה על חומרת קוונטית
Transpiling של מעגלים קוונטיים להרצה על חומרה בדרך כלל כרוך [במספר שלבים](/guides/transpiler-stages). בדרך כלל, השלבים הכרוכים בעומס חישובי רב ביותר הם בחירת פריסת הקיוביטים, ניתוב השערים הדו-קיוביטיים כדי להתאים לקישוריות הקיוביטים של החומרה, ואופטימיזציה של המעגל כדי למזער את ספירת השערים והעומק שלו. בפרוטוקול GEM, שלבי הפריסה והניתוב אינם נחוצים מכיוון שהקישוריות של החומרה כבר משולבת בעיצוב הפרוטוקול. למעגלים כבר יש פריסת קיוביטים, והשערים הדו-קיוביטיים כבר ממופים על חיבורים מקוריים. יותר מכך, כדי לשמר את המבנה של המעגל בזמן ששינוי זווית $R_{ZZ}$, יש לבצע רק אופטימיזציה מאוד בסיסית של המעגל.

המחלקה `GemExperiment` מבצעת transpile במעגלים בשקיפות בזמן ביצוע הניסוי. שלבי הפריסה והניתוב כבר מוחלפים כברירת מחדל כך שלא יעשו דבר, ואופטימיזציה של המעגל מבוצעת ברמה שמטפלת רק בשערים חד-קיוביטיים. עם זאת, ניתן לעקוף או להעביר אפשרויות נוספות באמצעות המתודה `set_transpile_options`. למטרות הדמיה, תא הקוד הבא מבצע transpile ידני למעגל שהוצג קודם לכן, ומצייר את המעגל ה-transpiled.

In [13]:
exp_data.figure("two_point_correlation")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/4ecb25c8-e572-49af-a879-9943039db131-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/e9b99d48-8d33-46b5-bff5-480ab1c1c1f2-0.avif)

## שלב 3: הרצה באמצעות Qiskit primitives
כדי להריץ את מעגלי פרוטוקול GEM על החומרה, קראו למתודה `run` של אובייקט `GemExperiment`. ניתן לציין את מספר ה-shots שברצונכם לדגום מכל מעגל. המתודה `run` מחזירה אובייקט [ExperimentData](https://qiskit-community.github.io/qiskit-experiments/stubs/qiskit_experiments.framework.ExperimentData.html) שכדאי לשמור במשתנה. שימו לב שהמתודה `run` רק מגישה עבודות מבלי לחכות שהן יסתיימו, ולכן זוהי קריאה לא חוסמת.

In [14]:
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/2b351d68-3924-445a-94ef-047b16214e8a-0.avif" alt="Output of the previous code cell" />

כדי לחכות לתוצאות, קראו למתודה `block_for_results` של אובייקט `ExperimentData`. קריאה זו תגרום למפרש להיתלות עד שהעבודות יסתיימו.

In [15]:
gem_exp = GemExperiment(
    plaquette_lattice.filter(range(3, 9)), backend=backend
)
gem_exp.set_experiment_options(schedule_idx=0)
exp_data = gem_exp.run(shots=10_000)
exp_data.block_for_results()
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/08581c09-a6a5-4a56-9fc4-abf22b063c6a-0.avif" alt="Output of the previous code cell" />

In [16]:
gem_exp = GemExperiment(plaquette_lattice, backend=backend)
gem_exp.set_experiment_options(schedule_idx=0)
exp_data = gem_exp.run(shots=10_000)
exp_data.block_for_results()
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/37e9a4cd-6efb-4ade-ad09-8139db9d58e9-0.avif" alt="Output of the previous code cell" />

## שלב 4: עיבוד לאחר והחזרת תוצאה בפורמט קלאסי רצוי
בזווית $R_{ZZ}$ של $\frac{\pi}{2}$, המצב המפוענח יהיה מצב GHZ בהיעדר רעש. ניתן לדמיין את הסדר ארוך הטווח של מצב GHZ על ידי שרטוט המגנטיזציה של מחרוזות הביטים הנמדדות. המגנטיזציה $M$ מוגדרת כסכום של אופרטורי Pauli $Z$ חד-קיוביטיים,
$$
M = \sum_{j=1}^N Z_j,
$$
כאשר $N$ הוא מספר קיוביטי האתר. הערך שלה עבור מחרוזת ביטים שווה להפרש בין מספר האפסים ומספר האחדים. מדידת מצב GHZ מניבה את מצב כל האפסים או את מצב כל האחדים בהסתברות שווה, כך שהמגנטיזציה תהיה $+N$ מחצית מהזמן ו-$-N$ את המחצית האחרת של הזמן. בנוכחות שגיאות עקב רעש, ערכים אחרים יופיעו גם כן, אך אם הרעש אינו גדול מדי, ההתפלגות עדיין תהיה מרוכזת ליד $+N$ ו-$-N$.

עבור מחרוזות הביטים הגולמיות לפני הפענוח, התפלגות המגנטיזציה תהיה שווה ערך לזו של מחרוזות ביטים אקראיות אחידות, בהיעדר רעש.

תא הקוד הבא מצייר את המגנטיזציה של מחרוזות הביטים הגולמיות ומחרוזות הביטים המפוענחות בזווית $R_{ZZ}$ של $\frac{\pi}{2}$.